# Real Estate Analysis - Mortgage Calculator

## Here we are going to calculate different features which are able to tell the details about the mortagage

### Please follow the steps thourogly for easy way to build the model

In [1]:
#importing the required libraries
import pandas as pd 
import numpy as np
from datetime import date

In [2]:
#assumption we make to find our final output
interest = 0.04
years = 30
payment_year = 12
mortagage = 400000
start_date = (date(2021,1,1))

In [4]:
#in the latest version of python np doesnot work for financial calculations , that is why we install 'numpy-financial'
pip install numpy-financial

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy_financial as npf

In [7]:
#pmt is nothing but to simplify the monthly payments
pmt = -1 * npf.pmt(interest/12, years*payment_year, mortagage)
pmt

1909.6611818618153

In [8]:
#monthly payments
pmt = -1 * npf.pmt(interest/12, years*payment_year, mortagage)

#Interest payment 
ipmt = -1* npf.ipmt(interest/payment_year, 1, years*payment_year, mortagage)

#principal payment 
ppmt = -1 * npf.ppmt(interest/payment_year, 1, years * payment_year, mortagage)

In [11]:
print(ipmt, ppmt, ppmt)

1333.3333333333335 576.3278485284818 576.3278485284818


In [13]:
#lest create the dataframe to implement the mortgage calculator

rng = pd.date_range(start_date, periods = years*payment_year, freq = 'MS')
rng.name = 'Payment date'

df = pd.DataFrame(index = rng, columns=['Payment', 'Principal paid', 'Interest paid','Ending Balance'], dtype='float')
df.reset_index(inplace=True)
df.index += 1
df.index.name = 'Period'

In [14]:
df

,Payment date,Payment,Principal paid,Interest paid,Ending Balance
Period,,,,,
1,2021-01-01,NaN,NaN,NaN,NaN
2,2021-02-01,NaN,NaN,NaN,NaN
3,2021-03-01,NaN,NaN,NaN,NaN
4,2021-04-01,NaN,NaN,NaN,NaN
5,2021-05-01,NaN,NaN,NaN,NaN
...,...,...,...,...,...
356,2050-08-01,NaN,NaN,NaN,NaN
357,2050-09-01,NaN,NaN,NaN,NaN
358,2050-10-01,NaN,NaN,NaN,NaN


In [36]:
#here we fill the values using the required formulae
df['Payment'] = -1* npf.pmt(interest/12, years*payment_year, mortagage)
df['Principal paid'] = -1*npf.ppmt(interest/payment_year,df.index, years* payment_year, mortagage)
df['Interest paid'] = -1*npf.ipmt(interest/payment_year, df.index, years*payment_year, mortagage)
df = df.round(2)

In [39]:
#this one cam beacuse i mistakenly added a wrong column 
del df1['Pricipal paid']

# 

In [38]:
#and this one for removing duplicate columns
df = df.T.drop_duplicates().T

In [40]:
df

,Payment date,Payment,Principal paid,Interest paid,Ending Balance
Period,,,,,
1,2021-01-01,1909.66,576.33,1333.33,NaN
2,2021-02-01,1909.66,578.25,1331.41,NaN
3,2021-03-01,1909.66,580.18,1329.48,NaN
4,2021-04-01,1909.66,582.11,1327.55,NaN
5,2021-05-01,1909.66,584.05,1325.61,NaN
...,...,...,...,...,...
356,2050-08-01,1909.66,1878.15,31.51,NaN
357,2050-09-01,1909.66,1884.41,25.25,NaN
358,2050-10-01,1909.66,1890.69,18.97,NaN


In [41]:
#calculating the ending balance for the first row
df['Ending Balance'] = 0
df.loc[1,'Ending Balance'] =  mortagage-df.loc[1,'Principal Paid']

In [42]:
df

,Payment date,Payment,Principal paid,Interest paid,Ending Balance
Period,,,,,
1,2021-01-01,1909.66,576.33,1333.33,399423.67
2,2021-02-01,1909.66,578.25,1331.41,0.00
3,2021-03-01,1909.66,580.18,1329.48,0.00
4,2021-04-01,1909.66,582.11,1327.55,0.00
5,2021-05-01,1909.66,584.05,1325.61,0.00
...,...,...,...,...,...
356,2050-08-01,1909.66,1878.15,31.51,0.00
357,2050-09-01,1909.66,1884.41,25.25,0.00
358,2050-10-01,1909.66,1890.69,18.97,0.00


In [43]:
#we are creating for loop to stop the recurring value for the 'Ending Balance'
for period in range(2, len(df)+1):
    previos_balance = df.loc[period-1, 'Ending Balance']
    principal_paid = df.loc[period, 'Principal paid']
    
    if previos_balance == 0:
        df.loc[period, ['Payment', 'Principal paid', 'Interest paid', 'Ending Balance']] == 0
        continue
    elif principal_paid <= previos_balance:
        df.loc[period, 'Ending Balance']= previos_balance - principal_paid
        

In [44]:
#finally we are able to calcuate the mortagage for 30yeard with loan amount 400000$
df

,Payment date,Payment,Principal paid,Interest paid,Ending Balance
Period,,,,,
1,2021-01-01,1909.66,576.33,1333.33,399423.67
2,2021-02-01,1909.66,578.25,1331.41,398845.42
3,2021-03-01,1909.66,580.18,1329.48,398265.24
4,2021-04-01,1909.66,582.11,1327.55,397683.13
5,2021-05-01,1909.66,584.05,1325.61,397099.08
...,...,...,...,...,...
356,2050-08-01,1909.66,1878.15,31.51,7575.42
357,2050-09-01,1909.66,1884.41,25.25,5691.01
358,2050-10-01,1909.66,1890.69,18.97,3800.32


### We can see process from above steps that we are able to calculate the different features that are required to find our mortgage. 

#### I am sure we can go ahead and deploy it a website which needs the mortgage calculator, cheers. 